In [1]:
from src.turbofan import Turbofan
from src.turboprop import Turboprop
from utils.configs import (
    config_ex71,
    config_ex72,
    config_ex73,
)

In [2]:
turboprop = Turboprop(config_ex71)
turboprop.set_sea_level_air_flow(8.49)
turboprop.print_outputs()
turboprop.calibrate_pot_th()
turboprop.print_outputs()

--- Resultados da Simulação do Motor ---

[ Estações do Motor ]
Estação              | Temp. Total (K)      | Pressão Total (kPa) 
------------------------------------------------------------------
2 (Inlet)            | 288.150              | 101.300             
3 (Compressor)       | 713.156              | 1597.501            
4 (Câmara)           | 1600.000             | 1597.501            
5 (Turbina Comp.)    | 1174.994             | 390.159             
6 (Turbina Livre)    | 931.033              | 135.575             

[ Dados da Turbina Livre ]
Razão de expansão na tl            : 2.878
Pressão total na saída da tl       : 135.58 kPa
Temperatura Isentrópica Total      : 903.93 K
Temperatura Total                  : 931.03 K
Cp_tl                              : 1.16
Trabalho Isentrópico               : 314.44 kJ/kg
Trabalho Real                      : 282.99 kJ/kg
Vazão na turbina                   : 8.68 kg/s
Potência na Turbina Livre          : 2457.33 kW
Potência na Gearbox

### Exemplo 7.2

In [25]:
turboprop.update_final_config(config_ex72)
turboprop.update_from_N2(0.85)
turboprop.print_outputs()

--- Resultados da Simulação do Motor ---

[ Estações do Motor ]
Estação              | Temp. Total (K)      | Pressão Total (kPa) 
------------------------------------------------------------------
2 (Inlet)            | 256.535              | 46.156              
3 (Compressor)       | 577.216              | 534.552             
4 (Câmara)           | 1310.669             | 534.552             
5 (Turbina Comp.)    | 989.988              | 144.472             
6 (Turbina Livre)    | 798.312              | 55.727              

[ Dados da Turbina Livre ]
Razão de expansão na tl            : 2.593
N2                                 : 0.850
Pressão total na saída da tl       : 55.73 kPa
Temperatura Isentrópica Total      : 781.59 K
Temperatura Total                  : 798.31 K
Cp_tl                              : 1.16
Trabalho Isentrópico               : 241.74 kJ/kg
Trabalho Real                      : 222.34 kJ/kg
Vazão na turbina                   : 2.85 kg/s
Potência na Turbina Livre

### Exemplo 7.3

In [19]:
turbofan = Turbofan(config_ex73)
turbofan.set_sea_level_air_flow(88.42)
turbofan.save_design_point()
turbofan.update_from_N2(0.85)
turbofan.get_thrust()

4.665384901750312

In [20]:
turbofan.print_outputs()

--- Resultados da Simulação do Motor ---

[ Estações do Motor ]
Estação         | Temp. Total (K)      | Pressão Total (kPa) 
-------------------------------------------------------------
2 (Inlet)       | 256.535              | 46.920              
8 (Fan)         | 278.273              | 61.028              
3 (Compressor)  | 512.314              | 373.335             
4 (Câmara)      | 1269.711             | 373.335             
5 (Turbina Comp.) | 1035.669             | 145.238             
6 (Turbina Fan) | 922.634              | 84.953              

[ Velocidades de Saída ]
Velocidade de Voo (u_0)         : 141.955 m/s
Velocidade Bocal Quente (u_core): 587.742 m/s
Velocidade Bocal Frio (u_fan)   : 243.148 m/s

[ Performance Geral ]
Razão Combustível/Ar (f)        : 0.01934
Empuxo Específico               : 0.96919 kN.s/kg
Consumo Específico (TSFC)       : 0.01995 kg/(kN.s)
N2                                 : 0.850
N1                                 : 0.799
Empuxo Total         

In [24]:
turbofan.sea_level_air_flow

61.68421249750001